In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# 미리 다운로드된 주식 데이터를 로드합니다.
data = pd.read_csv('stock_prices.csv')

# 수익률을 계산합니다.
returns = data.pct_change()

# 연간 평균 수익률을 계산합니다.
mean_returns = returns.mean() * 252

# 공분산 행렬을 계산합니다.
cov_matrix = returns.cov() * 252

# 가중치의 초기값을 설정합니다. 여기서는 동일 가중치를 사용합니다.
weights = np.full(mean_returns.shape[0], 1/mean_returns.shape[0])

# 샤프비율을 계산하는 함수를 정의합니다.
def calculate_sharpe_ratio(weights):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_vol
    return -sharpe_ratio  # 최적화를 위해 음수로 반환합니다.

# 가중치의 합이 1이 되도록 제약 조건을 설정합니다.
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# 가중치는 0과 1 사이의 값이 되도록 범위를 설정합니다.
bounds = tuple((0, 1) for asset in range(len(mean_returns)))

# 최적화 함수를 호출합니다.
optimal_weights = minimize(calculate_sharpe_ratio, weights, method='SLSQP', bounds=bounds, constraints=constraints)

# 최적화된 가중치를 출력합니다.
print('Optimal weights:', optimal_weights['x'])

# 최적화된 샤프비율을 출력합니다.
print('Optimized Sharpe ratio:', -optimal_weights['fun'])